In [ ]:
library(survminer)
library(survival)
library(survminer)
library(forestplot)
library(readxl)
library(dplyr)
library(gridExtra)
library(scales)
library(pROC) 
library(readxl) 
library(RColorBrewer)
library(forestploter)
library(extrafont)
loadfonts(device = "win")  

In [ ]:
result2 = read_excel("E:/0_pro/7.多中心ITH/1.数据分析/生存分析/本院生存分析/统计结果/本院+外部多因素Cox_RFS.xlsx",sheet = 'Sheet 1')
result2$Characteristics <- ifelse(is.na(result2$`HR (95% CI)`), 
                                  result2$Characteristics, 
                                  paste0("    ", result2$Characteristics))
# 处理Characteristics列，去除“Hypertension”前面的空格
result2 <- result2 %>%
  mutate(Characteristics = ifelse(Characteristics == "    Age", trimws(Characteristics), Characteristics))
# result2 <- result2 %>%
#   mutate(Characteristics = ifelse(Characteristics == "    Previous cardiovascular disease", trimws(Characteristics), Characteristics))

result2

In [ ]:
# 把NA替换为空字符串
# result2$`Regression Coefficient` <- ifelse(is.na(result2$`Regression Coefficient`), "", result2$`Regression Coefficient`)
# result2$`Points assigned` <- ifelse(is.na(result2$`Points assigned`), "", result2$`Points assigned`)
result2$`HR (95% CI)` <- ifelse(is.na(result2$`HR (95% CI)`), "", result2$`HR (95% CI)`)
# result2$`C` <- ifelse(is.na(result2$`C`), "", result2$`C`)
# result2$`P` <- ifelse(is.na(result2$`P`), "", result2$`P`)
result2$`HR` <- ifelse(is.na(result2$`HR`), "", result2$`HR`)
result2$`LowerCI` <- ifelse(is.na(result2$`LowerCI`), "", result2$`LowerCI`)
result2$`UpperCI` <- ifelse(is.na(result2$`UpperCI`), "", result2$`UpperCI`)
result2$`P value` <- ifelse(is.na(result2$`P value`), "", result2$`P value`)

In [ ]:
# 确保 UpperCI 和 HR 列为数值类型
result2$UpperCI <- as.numeric(result2$UpperCI)
result2$HR <- as.numeric(result2$HR)
# 计算标准误
result2$se <- (log(result2$UpperCI) - log(result2$HR))/1.96
# result2

# 为森林图添加空白列，为了产生一个绘图区间，用于显示CI
result2$` ` <- paste(rep(" ", 12), collapse = " ")
# result2

result2$LowerCI <- as.numeric(result2$LowerCI)
result2

In [ ]:
# 定义一个简单的主题，大家可以随意发挥自己的审美！
tm <- forest_theme(base_size = 10,           # 设置基础字体大小
                   ci_lwd = 1,          # 可信区间的线宽
                   refline_col = "red4",     # 设置参考线颜色为红色
                   arrow_type = "closed",    # 设置箭头类型为闭合箭头
                   ci_col = "blue4",    # 可信区间的边框颜色
#                    ci_fill = "blue4",      # 可信区间的填充颜色
                   footnote_col = "blue4")   # 设置脚注文字颜色为蓝色

# 绘制森林图
# p <- forest(result2[,c(1,2,10, 7,3)],   # 选择要在森林图中使用的数据列，这里包括变量名列、患者数量列、绘图要用的空白列和HR（95%CI）列
p <- forest(result2[,c(1,8, 2,3)],   # 选择要在森林图中使用的数据列，这里包括变量名列、患者数量列、绘图要用的空白列和HR（95%CI）列            
            est = result2$HR,          # 效应值，也就是HR列
            lower = result2$LowerCI,  # 置信区间下限
            upper = result2$UpperCI,  # 置信区间上限
#             sizes = result2$se,        # 黑框框的大小,自适应大小
            sizes = 0.3,        # 黑框框的大小            
            ci_column = 2,             # 在第3列（可信区间列）绘制森林图
            ref_line = 1,              # 添加参考线
            ci_lwd = 1,          # 可信区间的线宽
            
#             arrow_lab = c("Low risk", "High Risk"),  # 箭头标签，用来表示效应方向，如何设置取决于你的样本情况
            xlim = c(0, 9),          # 设置x轴范围
            ticks_at = c(0, 1, 3,5,7,9),  # 在指定位置添加刻度
            theme = tm,                # 添加自定义主题
            footnote = " ")  # 添加脚注信息
# 使用ggsave保存图像为PNG格式
# pp <- edit_plot(p, row = 10, gp = gpar(col = "red4", fontface = "italic")) #斜体
pp <- edit_plot(p, row = 10, gp = gpar(col = "red4"))
pp <- edit_plot(pp, row = 3, gp = gpar(col = "red4"))
pp
ggsave(filename = "E:/0_pro/7.多中心ITH/1.数据分析/生存分析/本院生存分析/统计结果/本院+外部_RFS_Cox森林图.png", 
       plot = pp, 
       device = "png", 
       width = 6, 
       height = 3, 
       units = "in", 
       dpi = 700, 
#        family = "Times New Roman",
       limitsize = FALSE,
       bg = "white")
pp